In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False

import numpy as np
import seaborn as sns

import re
import folium


In [6]:
csvList = ['cafeteria', 'delivery de comida', 'drive thru', 'padaria', 'restaurante']
dataFrames = []

for nome in csvList:
    # Lê o arquivo Bruto
    df = pd.read_csv(f"database/dadosBrutos/{nome}.csv", sep=';')

    # Dropa os registros duplicados em NOME e LOCAL deixando somente a primeira ocorrencia
    df = df.drop_duplicates(subset=['NOME', 'LOCAL'], keep='first')

    # Concatena os dados
    dataFrames.append(df)

# Cria um unico dataFrame Bruto
df_Bruto = pd.concat(dataFrames, ignore_index=True)

# Busco por ocorrencias de estabelecimentos que não são de alimentação
palavras_chave = [
    'restaurante', 'pizzaria', 'hamburgueria', 'cafeteria', 'doceria', 'confeitaria',
    'bar', 'delivery', 'sorveteria', 'padaria', 'comida', 'buffet', 'açaí', 'café',
    'carne', 'italiano', 'pizza', 'food', 'chá', '$', '$$'
]

# Cria um filtro para os que são drive thru e que batem com as palavras-chave em TIPO
filtro_drive_thru = (
    (df_Bruto['BUSCA_ID'] == 'drive thru') &
    (df_Bruto['TIPO'].fillna('').str.lower().apply(lambda x: any(palavra in x for palavra in palavras_chave)))
)

# Cria um filtro para os que NÃO são drive thru (mantém tudo)
filtro_outras = (df_Bruto['BUSCA_ID'] != 'drive thru')

# Combina os dois: mantém todas as outras linhas + só os drive thru filtrados
filtro_final = filtro_outras | filtro_drive_thru

df_filtrado = df_Bruto[filtro_final]
df_filtrado.to_csv("database/dadosTratados/bkp_df_filtrado.csv", index=False)


df_filtrado

,NOME,PONTUACAO,COMENTARIO,LOCAL,TIPO,TEMPO_ESPERA,BUSCA_ID
0,Shopping RioMar Fortaleza,4.7,52.356 comentários,"R. Des. Lauro Nogueira, 1500 - Papicu, Fortale...","Shopping center em Fortaleza, Ceará",45 min a 3 h,cafeteria
1,Greta Café,4.7,1.094 comentários,"Av. Antônio Sales, 2960 - Dionísio Torres, For...",$$,NaN,cafeteria
2,Bardo Café,4.9,22 comentários,"R. Amélia Benebien, 471 - Papicu, Fortaleza - ...",Cafeteria,NaN,cafeteria
3,"Go Coffee: Cafés, Chás, Frappes, Chocolate Que...",4.6,62 comentários,"R. Pereira Valente, 578 - Meireles, Fortaleza ...",Cafeteria,10 min,cafeteria
4,Le Pain Le Café | Aldeota,4.4,1.277 comentários,"Rua Professor Dias da Rocha, 670 - Meireles, F...",$$$,"20 min a 1,5 h",cafeteria
...,...,...,...,...,...,...,...
1059,Bolo,NaN,NaN,"Santa rosa - Santa Rosa, Fortaleza - CE, 60703...",NaN,NaN,restaurante
1060,Restaurante,3.0,Um comentário,"R. Gama, 170 - Vila Velha, Fortaleza - CE, 603...",Restaurante,NaN,restaurante
1061,Restaurante,NaN,NaN,"Rua Ministro Antônio Coelho - Vila Velha, Fort...",NaN,NaN,restaurante
1062,Pastel Quente,4.6,21 comentários,"Av. Duque de Caxias, 235 - Centro, Fortaleza -...",Pastelaria brasileira,NaN,restaurante


In [23]:
df_filtrado = df_filtrado.copy()

# Alteração dos nomes de coluna para melhor entendimento

# LOCAL PARA ENDERECO
df_filtrado = df_filtrado.rename(columns={'LOCAL': 'ENDERECO'})

# TIPO para LOCAL
df_filtrado = df_filtrado.rename(columns={'TIPO': 'LOCAL'})

# BUSCA_ID para TIPO
df_filtrado = df_filtrado.rename(columns={'BUSCA_ID': 'TIPO'})

In [24]:
# Quantidade de Informações NaN
nan = df_filtrado.isna().sum().to_frame(name='NaN Count')
nan['NaN %'] = (nan['NaN Count'] / len(df_filtrado)) * 100
nan['NaN %'] = nan['NaN %'].round(2)

print("Quantidade de Informações NaN")
nan

Quantidade de Informações NaN


,NaN Count,NaN %
NOME,0,0.00
PONTUACAO,226,25.28
COMENTARIO,226,25.28
ENDERECO,1,0.11
LOCAL,115,12.86
TEMPO_ESPERA,587,65.66
TIPO,0,0.00
